# BEFORE USING THIS NOTEBOOK
script written by Byron A. Adams - Updated: March 2021

## Description:
This script was primarily designed to explore landscape evolution in a region influenced by the depostion of spatially expanisve and thick volcanoclasitc deposits similar to ignimbrite deposits. This landscape evolution model assumes the erosion response will be detachment limited, and does not consider the affects of rainfall.
This script uses the FastScape eroder see the reference below for more details and to understand the input parameters 
Braun, J., Willett, S. (2013). A very efficient, implicit and parallel method to solve the stream power equation governing fluvial incision and landscape evolution Geomorphology,  181, 170-179.

## NOTES:
BLOCK 2 contains several input parameters that can be changed to adjust model results. This block contains the parameters that set the relief scale of the initial landscape and the pace and pattern of response after volcanic deposition.

BLOCK 4 contains the rock uplift rate field. this may also need to be changed.As there is only one rock uplift block, the rate does not change after volcanic depostion.

BLOCK 5 runs fastscape

The script outputs 2 netCDF files which contain the field values of the master grid for each time step. One file is produced for the initial landscape and then another for the volcanic landscape.

In [ ]:
## MISE EN PLACE! ## BLOCK 1
## Import math and graphics packages 
import numpy as np
import scipy as spy
import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr
import holoviews as hv
hv.notebook_extension('matplotlib')
import time

def backline():
    print('\r', end='')

## Import Landlab components
from landlab.components import DepressionFinderAndRouter, Lithology, LithoLayers
from landlab.components import FlowAccumulator, FastscapeEroder, ChiFinder

## Import Landlab utilities
from landlab import RasterModelGrid, imshow_grid
from landlab.io import read_esri_ascii

In [ ]:
## USER INPUT ## BLOCK 2
#Add simulation name for output data and set flag to print output
name = 'model_1'
print_flag = 'yes'

#Set time parameters
total_t_fsc = 4e6                # [yr] the duration of the model simluation
dt_fsc = 1e4                     # [yr] the time resolution for saving data

#Set erosion parameters
uplift_rate = 21.0e-4            # [m/yr]
K_br = 8.e-7                     # [units depend on n] rock erodibilty
m_sp = 1.0                       # [-] the area exponent relating A to discharge 
n_sp = 2.0                       # [-] the slope exponent

#Set rock parameters
thicknesses = [30000]            #Set the thickness of the bedrock (make sure there is enough...)
ids = [0]                        #Set the ID to 0, add more later
attrs = {'K_sp': {0: K_br, 1: 1.0e-5}}

In [ ]:
## ESTABLISH BOUNDARY CONDITIONS ## BLOCK 3
#Read in DEM
(mg, z) = read_esri_ascii("start_topo.txt", name="topographic__elevation")

#Set model boundary edges so that the all points drain to the bottom
mg.set_closed_boundaries_at_grid_edges(top_is_closed=True,bottom_is_closed=False,
                                       left_is_closed=True,right_is_closed=True)

#Add field 'bedrock elevation' to the grid
br = mg.add_zeros('bedrock__elevation',at='node')

#Add field ’rock uplift’ to the grid
U = mg.add_zeros('node','rock_uplift__rate')

#Add field ’erosion rate’ to the grid
E = mg.add_zeros('node','erosion__rate')

In [ ]:
## UPLIFT RATE SCENARIOS ## BLOCK 4
##Uniform rock uplift
U[mg.core_nodes] += uplift_rate

In [ ]:
## INITIAL STEADY-STATE LANDSCAPE ## BLOCK 5
#intitalize variables for plotting
zi = np.zeros(z.size)
fsc_steps = int(total_t_fsc/dt_fsc)

#Instantiate Landlab components
fr = FlowAccumulator(mg,flow_director = 'D8')
df = DepressionFinderAndRouter(mg)
cf = ChiFinder(mg,min_drainage_area=1.,reference_concavity=m_sp/n_sp)
fsc = FastscapeEroder(mg,K_sp = K_br,m_sp = m_sp,n_sp = n_sp,erode_flooded_nodes=False)

#Create a data structure for recording output using xarray
out_fields = ['topographic__elevation','erosion__rate','flow__receiver_node',
              'drainage_area','topographic__steepest_slope','channel__chi_index',
              'erosion__rate']
ds = xr.Dataset(
    data_vars={
        'topographic__elevation': (('time', 'y', 'x'),            # tuple of dimensions
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),      # n-d array of data
            {'units': 'm','long_name': 'Topographic Elevation'}), # dictionary with data attributes
        'erosion__rate': (('time', 'y', 'x'),
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),
            {'units': 'm/yr','long_name': 'Erosion Rate'}),
        'flow__receiver_node': (('time', 'y', 'x'),
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),
            {'units': '-','long_name': 'Flow direction (node)'}),
        'drainage_area': (('time', 'y', 'x'),
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),
            {'units': 'm**2','long_name': 'Drainage Area'}),
        'topographic__steepest_slope': (('time', 'y', 'x'),
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),
            {'units': '-','long_name': 'Slope'}),
        'channel__chi_index': (('time', 'y', 'x'),
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),
            {'units': '-','long_name': 'Chi'}),
        'erosion__rate': (('time', 'y', 'x'),
            np.empty((fsc_steps, mg.shape[0], mg.shape[1])),
            {'units': 'm/yr','long_name': 'Chi'})},
    coords={
        'x': (('x'),mg.x_of_node.reshape(mg.shape)[0, :],{'units': 'm'}), 
        'y': (('y'), mg.y_of_node.reshape(mg.shape)[:, 1], {'units': 'm'}),
        'time': (('time'), dt_fsc * np.arange(fsc_steps) / 1e6,{'units': 'Myr','standard_name': 'time'})})

start_time = time.time()

for i in range(fsc_steps):
    #Record the initial elevations of this step
    zi[:] = z[:]
    
    #Do some rock uplifting 
    mg.at_node['topographic__elevation'][:] += mg.at_node['rock_uplift__rate'][:] * dt_fsc
    
    fr.run_one_step()              #Run the flow router
    df.map_depressions()           #Run depression finder
    fsc.run_one_step(dt = dt_fsc)  #Run Fastscape
    cf.calculate_chi()             #Calculate a chi map
    
    #Calculate the erosion/deposition rate
    mg.at_node['erosion__rate'][:] = mg.at_node['rock_uplift__rate'][:] - ((mg.at_node['topographic__elevation'][:] - zi) / dt_fsc)
    
    #Record grid data for reanalysis
    for of in out_fields:
        ds[of][i, :, :] = mg['node'][of].reshape(mg.shape)
        
#save netCDF file
if print_flag == 'yes':
    filename = name + '_' + 'initial_landscape.nc'
    ds.to_netcdf(filename)